# Bài 10: Kết nối Pandas với SQL

Trong bài viết này, chúng ta sẽ cùng nhau tìm hiểu cách sử dụng MySQL server và kết nối với Pandas để thực hiện truy vấn dữ liệu. Nếu bạn nào mà làm việc với SQL rồi thì có thể sử dụng MSSQL có sẵn của mọi người, các bạn chưa học thì có thể dùng MySQL server.

## I. CÀI ĐẶT

- **Bước 1**: Cài đặt 1 số phần mềm sau:
    - Tải SQLServer tại [đây](https://dev.mysql.com/downloads/mysql/).
        - Với windows cài bản MSI Installer 435.7M. 
        - Với MacOS cài bản DMG Archive vì nó hỗ trợ UI. 
    - Tải MySQL Workbench tại [đây](https://dev.mysql.com/downloads/workbench/).
    - Tải database tại [đây](https://www.mediafire.com/folder/ajgkhko9x58ck/LEARNINGSQL_DB).

- **Bước 2**: Cài đặt thư viện hỗ trợ kết nối từ database đến Pandas thông qua câu lệnh 
```python 
!pip install mysql-connector-python
```

- **Bước 3**: Tạo database từ Python MySQL (tham khảo tại [đây](https://www.w3schools.com/python/python_mysql_create_db.asp)).
    - Click biểu tượng `create new schema`, nhập tên schema và click `Apply` x2 và `Close`. Khi đó trong phần `Schemas`  sẽ có thêm database mới có tên là `banking`.
    - Gõ `use banking` vào phần `Query`, khi đó DB banking sẽ được bôi đen lên.
    - Mở file `LearningSQL-MySQL-Script.sql` và chạy script đến bước 118 là thành công và refresh.
    
- **Bước 4**: Hiển thị EER diagram trong MySQL Workbench
    - Vào `Database` chọn `Reverse Engineering`
    - Chọn `Local instance 3306` trong mục `Stored connection` rồi chọn `Continue`
    - Nhập mật khẩu rồi chọn `Continue`
    - Chọn schema bằng cách tick vào ô `banking` rồi chọn lần lượt `Continue`, `Execute`, `Continue` và `OK`.
    
## II. THỰC HÀNH

### 1. Import thư viện

In [ ]:
import mysql.connector 
import pandas as pd
import numpy as np

### 2. Tạo 1 kết nối tới database

In [1]:
connection = mysql.connector.connect(
      host="localhost",
      user="root", 
      password="@Mci1234",
      database="banking"
) 

### 3. Truy vấn dữ liệu

#### **Câu hỏi:** Lấy tất cả các bản ghi trong bảng `ACCOUNT`

In [4]:
sql = 'SELECT * FROM ACCOUNT'
account = pd.read_sql(sql, connection)

#### **Câu hỏi:** Lấy tất cả các thông tin cột `ACCOUNT_ID` và `AVAIL_BALANCE` trong bảng `ACCOUNT`

In [8]:
sql = "SELECT ACCOUNT_ID, AVAIL_BALANCE FROM ACCOUNT"
account = pd.read_sql(sql,connection)

,ACCOUNT_ID,AVAIL_BALANCE
0,1,1057.75
1,2,500.00


In [9]:
sql = "SELECT * FROM ACCOUNT"
account = pd.read_sql(sql,connection)[['ACCOUNT_ID', 'AVAIL_BALANCE']]

,ACCOUNT_ID,AVAIL_BALANCE
0,1,1057.75
1,2,500.00


#### **Câu hỏi:** Liệt kê tên các bảng trong database `banking`

In [10]:
sql = "SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = 'banking'"
table_name = pd.read_sql(sql, connection)

,TABLE_NAME
0,ACC_TRANSACTION
1,ACCOUNT
2,BRANCH
3,BUSINESS
4,CUSTOMER
5,DEPARTMENT
6,EMPLOYEE
7,INDIVIDUAL
8,OFFICER
9,PRODUCT


#### **Câu hỏi:** Top 3 khách hàng có số dư cao nhất

In [11]:
sql = "SELECT c.CUST_ID, AVAIL_BALANCE FROM CUSTOMER c \
        LEFT JOIN ACCOUNT a ON c.CUST_ID = a.CUST_ID ORDER BY AVAIL_BALANCE DESC LIMIT 3"
account = pd.read_sql(sql, connection)

,CUST_ID,AVAIL_BALANCE
0,13,50000.0
1,12,38552.1
2,10,23575.1


In [12]:
sql = "SELECT * FROM ACCOUNT"
account_df = pd.read_sql(sql, connection)

sql = "SELECT * FROM CUSTOMER"
customer_df = pd.read_sql(sql, connection)

acccount = account_df.merge(customer_df, on='CUST_ID', how='inner')[['CUST_ID', 'AVAIL_BALANCE']].\
        sort_values('AVAIL_BALANCE', ascending=False).head(3)

,CUST_ID,AVAIL_BALANCE
23,13,50000.0
22,12,38552.1
19,10,23575.1


#### **Câu hỏi:** Số dư của khách hàng

In [13]:
sql = "SELECT a.cust_id, avail_balance FROM account a \
            INNER JOIN customer c ON a.cust_id = c.cust_id"
balance = pd.read_sql(sql, connection)

,cust_id,avail_balance
0,1,1057.75
1,1,500.00
2,1,3000.00
3,2,2258.02
4,2,200.00


In [14]:
sql = "SELECT a.cust_id, SUM(avail_balance) AS sum_balance FROM account a \
            INNER JOIN customer c ON a.cust_id = c.cust_id GROUP BY a.cust_id"
sum_balance = pd.read_sql(sql, connection)

,cust_id,sum_balance
0,1,4557.750000
1,2,2458.020020
2,3,3270.250000
3,4,6788.979858
4,5,2237.969971


Lưu ý, sau khi sử dụng xong, ta sử dụng câu lệnh `.close()` để ngắt kết nối với database.

In [16]:
connection.close()

Tham khảo thêm về tạo Database trong MySQL tại [MySQL Create DB](https://www.w3schools.com/python/python_mysql_create_db.asp)